In [1]:
import pandas as pd
import requests
from tqdm import tqdm
import random
import json
import time
import pickle
from selenium import webdriver
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from datetime import datetime
import re

In [2]:
df = pd.read_excel("wizzair_replies.xlsx")
df.head()

,text,favorited,favoriteCount,replyToSN,created,truncated,replyToSID,id,replyToUID,statusSource,screenName,retweetCount,isRetweet,retweeted,longitude,latitude
0,"@hankstwit Hi again, the refunds will be proce...",False,0,hankstwit,2020-05-01 13:22:52,True,1.255938e+18,1256212495401975809,1.113826e+08,"<a href=""https://www.hootsuite.com"" rel=""nofol...",wizzair,0,False,False,NaN,NaN
1,@TimLister9 We're always sending notifications...,False,0,TimLister9,2020-05-01 13:19:34,True,1.255938e+18,1256211665206599681,1.007237e+18,"<a href=""https://www.hootsuite.com"" rel=""nofol...",wizzair,0,False,False,NaN,NaN
2,@Stoyan66200747 Maybe the aircraft is upgraded...,False,0,Stoyan66200747,2020-05-01 13:11:33,False,1.255917e+18,1256209646123053056,1.255189e+18,"<a href=""https://www.hootsuite.com"" rel=""nofol...",wizzair,0,False,False,NaN,NaN
3,@inmotionslb Open the travel plan for your boo...,False,0,inmotionslb,2020-05-01 13:09:19,True,1.255882e+18,1256209083704709120,6.289613e+07,"<a href=""https://www.hootsuite.com"" rel=""nofol...",wizzair,0,False,False,NaN,NaN
4,@natkoc_ When cancelling the booking you shoul...,False,0,natkoc_,2020-05-01 13:09:12,True,1.255859e+18,1256209056202739713,6.120112e+08,"<a href=""https://www.hootsuite.com"" rel=""nofol...",wizzair,0,False,False,NaN,NaN


## Do NOT run this part

In [12]:
# I have not found a single twitter package (R or Python) that I could use to get the data for a specific conversation
# So I traced back the network communication as 
# I wanted to retrieve data for sg that is called a "conversation" in Twitter convention.
# This is the request that is sent out to get data for a specific conversation
# Btw https://curl.trillworks.com/ this is a very cool website to turn curl to python.

def get_json_for_conversation(conversation_id, x_csrf_token, x_guest_token, authorization):

    headers = {
    'x-twitter-client-language': 'en-GB',
    'x-csrf-token': f'{x_csrf_token}',
    'x-guest-token': f'{x_guest_token}',
    'x-twitter-active-user': 'yes',
    'authorization': f'{authorization}',
    }
    
    params = (
        ('include_profile_interstitial_type', '1'),
        ('include_blocking', '1'),
        ('include_blocked_by', '1'),
        ('include_followed_by', '1'),
        ('include_want_retweets', '1'),
        ('include_mute_edge', '1'),
        ('include_can_dm', '1'),
        ('include_can_media_tag', '1'),
        ('skip_status', '1'),
        ('cards_platform', 'Web-12'),
        ('include_cards', '1'),
        ('include_composer_source', 'true'),
        ('include_ext_alt_text', 'true'),
        ('include_reply_count', '1'),
        ('tweet_mode', 'extended'),
        ('include_entities', 'true'),
        ('include_user_entities', 'true'),
        ('include_ext_media_color', 'true'),
        ('include_ext_media_availability', 'true'),
        ('send_error_codes', 'true'),
        ('simple_quoted_tweets', 'true'),
        ('count', '20'),
        ('ext', 'mediaStats,highlightedLabel,cameraMoment'),
    )
    
    # I know I should oauth somehow myself as it is clearly an API it just doesn't seem to be publicly available.
    response = requests.get(f'https://api.twitter.com/2/timeline/conversation/{conversation_id}.json', 
                            headers = headers, 
                            params=params)
    if response.status_code == 200:
        return response.json()
    else:
        print(response.status_code)

In [13]:
# This function is to generate guest tokens for us. 
# Every time you open twitter you get a guest token, we are capitalising on this.
# One token usually can be used slightly more than a 100 times. 
# Each time the token is to expire, we just open twitter and get a new one.
def get_new_tokens():
    caps = DesiredCapabilities.CHROME
    caps['loggingPrefs'] = {'performance': 'ALL'}
    driver = webdriver.Chrome(desired_capabilities = caps) # chromedriver.exe needs to be placed in the same directory as the code.
    driver.get('https://twitter.com/inmotionslb/status/1255777964802932737') # This is just random, could be any conversation.
    time.sleep(5) # Selenium is very capricious, we better a few seconds to make sure it has loaded the page.
    
    # Parsing out the tokens from the network log. This is quite dirty stuff. 
    browser_log = driver.get_log('performance') 
    events = [json.loads(entry['message'])['message'] for entry in browser_log]
    relevant_requests = [event["params"]["request"] for event in events if "request" in event["params"].keys()]
    relevant_headers = [request["headers"] for request in relevant_requests if "x-guest-token" in request["headers"].keys()]
    random_header = random.choice(relevant_headers) # It doesn't matter which one you choose. All of them have the same tokens.
    time.sleep(5) # Again, capricious.
    
    driver.quit()
    return (random_header["x-csrf-token"], random_header["x-guest-token"], random_header["authorization"])

In [7]:
json_list = []
for counter, conversation_id in enumerate(tqdm(df["id"])):
    # 100 is based on historical experiments. The expiration of the guest token is slightly more than that.
    # However, I do not know the exact number because it varied a lot.
    # This is a safe call.
    if counter % 100 == 0: # If about to reach the limit, we just get a new guest token.
        x_csrf_token, x_guest_token, authorization = get_new_tokens()
    json_list.append(get_json_for_conversation(conversation_id, # Appending the response of the API to a list.
                                               x_csrf_token, 
                                               x_guest_token, 
                                               authorization))

100%|██████████████████████████████████████████████████████████████████████████████| 3198/3198 [42:08<00:00,  1.26it/s]


In [9]:
with open('wizzair_conversations.pickle', 'wb') as f:
    pickle.dump(json_list, f, protocol = pickle.HIGHEST_PROTOCOL)

## Go on running from here

In [3]:
with open(r"wizzair_conversations.pickle", "rb") as f:
    conversations = pickle.load(f)

In [4]:
def json_parser(input_json):
    features = [
        "created_at",
        "id_str",
        "full_text",
        "entities",
        "source",
        "conversation_id_str",
        "user_id_str",
        "in_reply_to_screen_name",
        "in_reply_to_user_id_str"
    ]
    output_json = {k: v for k, v in input_json.items() if k in features}
    try:
        output_json["entities"] = [mention["screen_name"] for mention in output_json["entities"]["user_mentions"]]
    except KeyError:
        output_json["entities"] = []
    output_json["created_at"] = datetime.strptime(output_json["created_at"], '%a %b %d %H:%M:%S +0000 %Y')
    output_json["source"] = re.compile(r'<.*?>').sub("", output_json["source"])
    
    return output_json

In [5]:
conversation_list = []
for conversation in conversations:
    for tweet_number in conversation["globalObjects"]["tweets"].keys():
        conversation_list.append(json_parser(conversation["globalObjects"]["tweets"][tweet_number]))

In [6]:
df = pd.DataFrame(conversation_list)
# creating a column indicating whether tweet is related to wizzair or else
df["is_customer"] = ["wizzair" if screen_name != "wizzair" else "customer" for screen_name in df["in_reply_to_screen_name"]]
# deleting mentions
df["full_text"] = [" ".join(filter(lambda word:word[0]!='@', tweet.split())) for tweet in df["full_text"]]
# deleting url-s
df["full_text"].replace("https\S+", "", regex = True, inplace = True)
df.head()

,created_at,id_str,full_text,entities,source,in_reply_to_user_id_str,in_reply_to_screen_name,user_id_str,conversation_id_str,is_customer
0,2020-05-02 09:41:01,1256519050735296512,The membership is available one year and as we...,[hankstwit],Hootsuite Inc.,111382627,hankstwit,624837541,1255938311216037899,wizzair
1,2020-05-01 14:08:27,1256223967368675330,You call they tell you go online. You charge f...,[wizzair],Twitter for Android,624837541,wizzair,111382627,1255938311216037899,customer
2,2020-04-30 19:13:22,1255938311216037899,"flight cancelled 3 times now, want my money ba...",[wizzair],Twitter for Android,624837541,wizzair,111382627,1255938311216037899,customer
3,2020-05-01 13:22:52,1256212495401975809,"Hi again, the refunds will be processed within...",[hankstwit],Hootsuite Inc.,111382627,hankstwit,624837541,1255938311216037899,wizzair
4,2020-04-30 19:10:13,1255937519583100929,You didn’t notify me of my cancelled flight!! ...,"[wizzair, JamesATpost]",Twitter for iPhone,624837541,wizzair,1007237383790657542,1255778326632968198,customer


In [7]:
# Having a look at the paltforms where messages are sent from.
df["source"].value_counts()

Hootsuite Inc.             5554
Twitter for iPhone         2768
Twitter Web App            2158
Twitter for Android        1542
Twitter for iPad             93
Twitter for Advertisers      55
Tweetbot for iΟS             43
TweetDeck                    42
Socialbakers                 26
Twitter Web Client           10
Twitterrific for iOS         10
Tweetbot for Mac              6
Echofon                       4
Twitter for Mac               3
Conversocial                  2
SocialElephants               2
Plume for Android             1
Twitter Media Studio          1
Name: source, dtype: int64

Hootsuite is a social media marketing & management platform. They have a service related to [handling twitter ](https://hootsuite.com/twitter) data as well. I am not sure whether answering the tweets is outsourced to this company or they just provide a platform to handle all the messages and WizzAir employs people to use this platform. But, answering this question is not key to my hypotheses.

In [8]:
# Just to make sure there is no funky thing going on.
df["in_reply_to_screen_name"].value_counts()

wizzair           5912
AlexisFerraris      40
ohmightycoffee      36
Lim03B              33
lucasfont95         32
                  ... 
babymoth1            1
mattkurton           1
Daniel_Simons        1
courts__xo           1
jockulla             1
Name: in_reply_to_screen_name, Length: 1968, dtype: int64

In [10]:
df.to_csv("tweets_formatted.csv", sep = ";")